# Multilayer perceptron in TensorFlow (using Keras)

In this script we build a small multilayer perceptron with two hidden layers having 500 and 50 neurons each for classifying the MNIST database of handwritten digits using Keras.

In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as imgplot
import numpy as np
# To be compatible with python3
try:
    import cPickle as pickle
except ImportError:
    import pickle
import gzip
import time
import tensorflow as tf
tf.set_random_seed(1)

from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
import keras
keras.__version__, tf.__version__

Using TensorFlow backend.


('1.2.2', '1.0.0')

In [2]:
with gzip.open('../../lasagne/mnist_4000.pkl.gz', 'rb') as f:
    (X,y) = pickle.load(f, encoding='latin1')
PIXELS = len(X[0,0,0,:])

print(X.shape, y.shape, PIXELS) #As read

# We need to reshape for the MLP
X = X.reshape([4000, 784])
np.shape(X)

(4000, 1, 28, 28) (4000,) 28


(4000, 784)

In [3]:
import tensorflow as tf
sess = tf.Session()

from keras import backend as K
K.set_session(sess)

In [4]:
# Taken from http://stackoverflow.com/questions/29831489/numpy-1-hot-array
def convertToOneHot(vector, num_classes=None):
    result = np.zeros((len(vector), num_classes), dtype='int32')
    result[np.arange(len(vector)), vector] = 1
    return result

In [ ]:
lambd = 0.000005
from keras.regularizers import l2, activity_l2
model = Sequential()
model.add(Dense(500, batch_input_shape=(None, 784),activity_regularizer=activity_l2(lambd)))
model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(50,activity_regularizer=activity_l2(lambd)))
model.add(Dropout(0.3))
#model.add(keras.layers.normalization.BatchNormalization())
model.add(Activation('relu'))

model.add(Dense(10, activation='softmax',activity_regularizer=activity_l2(lambd)))
model.compile(loss='categorical_crossentropy',
              optimizer='adadelta',
              metrics=['accuracy'])

In [ ]:
#tensorboard = keras.callbacks.TensorBoard(log_dir='/tmp/dumm/run4', write_graph=True)
history = model.fit(X[0:2400], 
          convertToOneHot(y[0:2400],10), 
          nb_epoch=10, 
          batch_size=128, 
          #callbacks=[tensorboard],
          validation_data=[X[2400:3000], convertToOneHot(y[2400:3000],10)])

Train on 2400 samples, validate on 600 samples
Epoch 1/10
2048/2400 [========================>.....] - ETA: 0s - loss: 2.2658 - acc: 0.3706

In [ ]:
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['acc', 'val_acc', 'loss', 'val_loss'],loc=10)
plt.ylim((0,1.1))